# Transforming data from bronze and writing to silver 

In [0]:
dbutils.fs.ls("/mnt/bronze/public/")

[FileInfo(path='dbfs:/mnt/bronze/public/customers/', name='customers/', size=0, modificationTime=1717946581000),
 FileInfo(path='dbfs:/mnt/bronze/public/orders/', name='orders/', size=0, modificationTime=1717946582000),
 FileInfo(path='dbfs:/mnt/bronze/public/products/', name='products/', size=0, modificationTime=1717946583000)]

In [0]:
dbutils.fs.ls("/mnt/silver/")

[]

In [0]:
table_names = [table.name.split('/')[0] for table in dbutils.fs.ls("mnt/bronze/public/")]

print(table_names)


['customers', 'orders', 'products']


In [0]:
for table in table_names:
    input_path = f"/mnt/bronze/public/{table}/{table}.parquet"
    df = spark.read.format('parquet').load(input_path)
    if table == "products":
        from pyspark.sql.functions import col
        df = df.withColumn("price", col("price") * 1.04)

    output_path = f"/mnt/silver/public/{table}/"
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
df.select("product_id", "price").show(5)

+----------+--------+
|product_id|   price|
+----------+--------+
|         1|766.9064|
|         2|351.2704|
|         3|  555.88|
|         4|462.5296|
|         5|999.5024|
+----------+--------+
only showing top 5 rows

